Creative Commons CC BY 4.0 Lynd Bacon & Associates, Ltd. Not warranted to be suitable for any particular purpose. (You're on your own!)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<h1 align='center'>Assignment 2: Random Forest Regression Example</h1>

In what follows we'll train a Random Forest (RF) regressor to predict the _Sale_Price_ target variable.  We'll use a selection of unscaled (e.g., not minmax or standardized rescaled) features.  We'll do a _grid search_ to examine the effects of different hyperparameter values.   We'll apply "dummy," or "one hot," encoding of a categorical variable as an example of encoding a discrete feature.

Note that it's up to you to choose the features used to train your models.

## Packages

Stuff you may need.

In [2]:
import numpy as np
import pandas as pd
import os
from pickleshare import *
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

## Data

The data are a subset of of the assignment data.

In [3]:
os.listdir()

['.ipynb_checkpoints',
 '2-Assignment-2-Guide-v1.ipynb',
 'ames-data-info.zip',
 'amesDF.pickle',
 'amesNumDFclean.pickle',
 'amesSelDF.pickle',
 'BenPrescott_Assignment2.ipynb',
 'data-input-select-ex-assignment-2-v1.ipynb',
 'DataDocumentation.txt',
 'decock.pdf',
 'kmeans-assignment-2-ex-v2.ipynb',
 'NAME.docx',
 'rescaling-PCA-ex-assignment-2-v1.ipynb',
 'RF-example-v1.ipynb',
 'saved_notebook.db']

In [4]:
aDF=pd.read_pickle('amesNumDFclean.pickle')

In [5]:
print(aDF.describe())

       Lot_Frontage       Lot_Area   Year_Built  Year_Remod_Add  Mas_Vnr_Area  \
count   2930.000000    2930.000000  2930.000000     2930.000000   2930.000000   
mean      57.647782   10147.921843  1971.356314     1984.266553    101.096928   
std       33.499441    7880.017759    30.245361       20.860286    178.634545   
min        0.000000    1300.000000  1872.000000     1950.000000      0.000000   
25%       43.000000    7440.250000  1954.000000     1965.000000      0.000000   
50%       63.000000    9436.500000  1973.000000     1993.000000      0.000000   
75%       78.000000   11555.250000  2001.000000     2004.000000    162.750000   
max      313.000000  215245.000000  2010.000000     2010.000000   1600.000000   

       BsmtFin_SF_1  BsmtFin_SF_2  Bsmt_Unf_SF  Total_Bsmt_SF  First_Flr_SF  \
count   2930.000000   2930.000000  2930.000000    2930.000000   2930.000000   
mean       4.177474     49.705461   559.071672    1051.255631   1159.557679   
std        2.233372    169.142089

In [6]:
# Missing values?

aDF.isna().sum().sum()

0

## Scrubbing, Feature Selection, Transformation

As per the data documentation, we're going to remove all data for homes with > 4,000 SQ FT.

A ubiquitous ML task.  Here there are a couple of interesting questions to be addressed.  One is how should the _year_ features be dealt with?  The data are up through 2010.  We could subtract each year value from 2010, which would change them to years since 2010.  Note that for the _year_ features to make sense, they should be logically related to each other.  For example, _Year_Built_ not be "later" than _Year_Sold_ or "later" than _Year_Remod_Add_.  It's reasonable to expect than any of these features my be predictive of _Sale_Price_, but they need to be expressed in a valid way.

The same goes for features that are essentially categorical, like _Mo_Sold_.  This feature needs to be recoded.  We'll do that. But first we'll do some checks on the _years_ features.

In [7]:
# As per the recommendation in the data documentation

aDF2=aDF[aDF.Gr_Liv_Area<4000].copy()    #What's the copy(), for?
aDF2.shape

(2925, 34)

### Weird Years

In [8]:
# Any homes sold before they are built?

((aDF2.Year_Built-aDF2.Year_Sold)>0.00).value_counts()

False    2925
dtype: int64

In [9]:
# Any homes remodelled before they are built?  One?

((aDF2.Year_Built-aDF2.Year_Remod_Add)>0.00).value_counts()

False    2924
True        1
dtype: int64

In [10]:
# Any remodels after sold?  One

((aDF2.Year_Remod_Add-aDF2.Year_Sold)>0.00).value_counts()

False    2924
True        1
dtype: int64

Let's get rid of the two (maybe just one) records with the anomalous year values.

In [11]:
aDF3=aDF2.loc[~((aDF2.Year_Built-aDF2.Year_Remod_Add)>0.00),:].copy()
aDF3.shape

(2924, 34)

In [12]:
aDF4=aDF3.loc[~((aDF2.Year_Remod_Add-aDF2.Year_Sold)>0.00),:].copy()
aDF4.shape

(2923, 34)

### Note

The "cleanliness" of the data you train your models on is your responsibility.  Data "hygiene" is in your hands!

### Recoding the _Mo_Sold_ feature

This feature is coded 1 through 12, but it's not _numeric_ per se, in the sense that the digits are values on a continuous metric.  Well, maybe it's a ranking.  In any case, here we're going to recoded it by dummy coding, or "one hot" encoding.  This will give use 12 features that are each coded in 0's and 1's.  You can do this using .get_dummies() in Pandas, or with the OneHotEncoder in scikit-learn.  We'll use the former, here.

In [13]:
moDummies=pd.get_dummies(aDF4.Mo_Sold,prefix="sales_mo").astype(int)
moDummies.columns

Index(['sales_mo_1.0', 'sales_mo_2.0', 'sales_mo_3.0', 'sales_mo_4.0',
       'sales_mo_5.0', 'sales_mo_6.0', 'sales_mo_7.0', 'sales_mo_8.0',
       'sales_mo_9.0', 'sales_mo_10.0', 'sales_mo_11.0', 'sales_mo_12.0'],
      dtype='object')

Next we'll remove _Sales_Month_ from our DataFrame, and then we'll add moDummies.

In [14]:
aDF5=aDF4.loc[:,~(aDF4.columns.isin(['Sale_Month']))]
aDF6=pd.concat([aDF5,moDummies],axis=1,ignore_index=False)
aDF6.columns

Index(['Lot_Frontage', 'Lot_Area', 'Year_Built', 'Year_Remod_Add',
       'Mas_Vnr_Area', 'BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF',
       'Total_Bsmt_SF', 'First_Flr_SF', 'Second_Flr_SF', 'Gr_Liv_Area',
       'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath', 'Half_Bath',
       'Bedroom_AbvGr', 'Kitchen_AbvGr', 'TotRms_AbvGrd', 'Fireplaces',
       'Garage_Cars', 'Garage_Area', 'Wood_Deck_SF', 'Open_Porch_SF',
       'Enclosed_Porch', 'Three_season_porch', 'Screen_Porch', 'Pool_Area',
       'Misc_Val', 'Mo_Sold', 'Year_Sold', 'Sale_Price', 'Longitude',
       'Latitude', 'sales_mo_1.0', 'sales_mo_2.0', 'sales_mo_3.0',
       'sales_mo_4.0', 'sales_mo_5.0', 'sales_mo_6.0', 'sales_mo_7.0',
       'sales_mo_8.0', 'sales_mo_9.0', 'sales_mo_10.0', 'sales_mo_11.0',
       'sales_mo_12.0'],
      dtype='object')

In [15]:
aDF6

,Lot_Frontage,Lot_Area,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,...,sales_mo_3.0,sales_mo_4.0,sales_mo_5.0,sales_mo_6.0,sales_mo_7.0,sales_mo_8.0,sales_mo_9.0,sales_mo_10.0,sales_mo_11.0,sales_mo_12.0
0,141.0,31770.0,1960.0,1960.0,112.0,2.0,0.0,441.0,1080.0,1656.0,...,0,0,1,0,0,0,0,0,0,0
1,80.0,11622.0,1961.0,1961.0,0.0,6.0,144.0,270.0,882.0,896.0,...,0,0,0,1,0,0,0,0,0,0
2,81.0,14267.0,1958.0,1958.0,108.0,1.0,0.0,406.0,1329.0,1329.0,...,0,0,0,1,0,0,0,0,0,0
3,93.0,11160.0,1968.0,1968.0,0.0,1.0,0.0,1045.0,2110.0,2110.0,...,0,1,0,0,0,0,0,0,0,0
4,74.0,13830.0,1997.0,1998.0,0.0,3.0,0.0,137.0,928.0,928.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,37.0,7937.0,1984.0,1984.0,0.0,3.0,0.0,184.0,1003.0,1003.0,...,1,0,0,0,0,0,0,0,0,0
2926,0.0,8885.0,1983.0,1983.0,0.0,2.0,324.0,239.0,864.0,902.0,...,0,0,0,1,0,0,0,0,0,0
2927,62.0,10441.0,1992.0,1992.0,0.0,3.0,0.0,575.0,912.0,970.0,...,0,0,0,0,1,0,0,0,0,0
2928,77.0,10010.0,1974.0,1975.0,0.0,1.0,123.0,195.0,1389.0,1389.0,...,0,1,0,0,0,0,0,0,0,0


In [16]:
#checking final data types

aDF6.dtypes.value_counts()

float64    34
int32      12
dtype: int64

### Transforming year features

Let's change each year feature so that it indicates years before 2010.  For ease of exposition, we'll first put the year features into a separate DataFrame, removing them from where they are in aDF6.  We'll transform them, and then put them back into a new DataFrame.

In [17]:
years=['Year_Built','Year_Remod_Add','Year_Sold']
yearsDF=aDF6.loc[:,years]
aDF7=aDF6.loc[:,~(aDF6.columns.isin(years))]

In [18]:
aDF7

,Lot_Frontage,Lot_Area,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,...,sales_mo_3.0,sales_mo_4.0,sales_mo_5.0,sales_mo_6.0,sales_mo_7.0,sales_mo_8.0,sales_mo_9.0,sales_mo_10.0,sales_mo_11.0,sales_mo_12.0
0,141.0,31770.0,112.0,2.0,0.0,441.0,1080.0,1656.0,0.0,1656.0,...,0,0,1,0,0,0,0,0,0,0
1,80.0,11622.0,0.0,6.0,144.0,270.0,882.0,896.0,0.0,896.0,...,0,0,0,1,0,0,0,0,0,0
2,81.0,14267.0,108.0,1.0,0.0,406.0,1329.0,1329.0,0.0,1329.0,...,0,0,0,1,0,0,0,0,0,0
3,93.0,11160.0,0.0,1.0,0.0,1045.0,2110.0,2110.0,0.0,2110.0,...,0,1,0,0,0,0,0,0,0,0
4,74.0,13830.0,0.0,3.0,0.0,137.0,928.0,928.0,701.0,1629.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,37.0,7937.0,0.0,3.0,0.0,184.0,1003.0,1003.0,0.0,1003.0,...,1,0,0,0,0,0,0,0,0,0
2926,0.0,8885.0,0.0,2.0,324.0,239.0,864.0,902.0,0.0,902.0,...,0,0,0,1,0,0,0,0,0,0
2927,62.0,10441.0,0.0,3.0,0.0,575.0,912.0,970.0,0.0,970.0,...,0,0,0,0,1,0,0,0,0,0
2928,77.0,10010.0,0.0,1.0,123.0,195.0,1389.0,1389.0,0.0,1389.0,...,0,1,0,0,0,0,0,0,0,0


In [19]:
yearsDF=yearsDF.transform(lambda x: 2010-x)
yearsDF.head()

,Year_Built,Year_Remod_Add,Year_Sold
0,50.0,50.0,0.0
1,49.0,49.0,0.0
2,52.0,52.0,0.0
3,42.0,42.0,0.0
4,13.0,12.0,0.0


In [20]:
aDF8=pd.concat([aDF7,yearsDF],axis=1)
aDF8.columns

Index(['Lot_Frontage', 'Lot_Area', 'Mas_Vnr_Area', 'BsmtFin_SF_1',
       'BsmtFin_SF_2', 'Bsmt_Unf_SF', 'Total_Bsmt_SF', 'First_Flr_SF',
       'Second_Flr_SF', 'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath',
       'Full_Bath', 'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr',
       'TotRms_AbvGrd', 'Fireplaces', 'Garage_Cars', 'Garage_Area',
       'Wood_Deck_SF', 'Open_Porch_SF', 'Enclosed_Porch', 'Three_season_porch',
       'Screen_Porch', 'Pool_Area', 'Misc_Val', 'Mo_Sold', 'Sale_Price',
       'Longitude', 'Latitude', 'sales_mo_1.0', 'sales_mo_2.0', 'sales_mo_3.0',
       'sales_mo_4.0', 'sales_mo_5.0', 'sales_mo_6.0', 'sales_mo_7.0',
       'sales_mo_8.0', 'sales_mo_9.0', 'sales_mo_10.0', 'sales_mo_11.0',
       'sales_mo_12.0', 'Year_Built', 'Year_Remod_Add', 'Year_Sold'],
      dtype='object')

## Training and Test Data

We'll set aside a randomly selected 20% of of the data in aDF8 to use as final test data.  We'll first separate our y target, _Sale_Price_ from the other variables, and we'll select some features to use in training our RF model.  

In [21]:
y=aDF8.Sale_Price.to_numpy(copy=True)  # extract np array from the DataFrame
SFfeats=aDF8.columns.to_list()[:10]   #SFfeatures
salesFeats=list(filter(lambda s: s.startswith('sales_'),
                       aDF8.columns.to_list()))   # sales month feature

RFfeatures=SFfeats+salesFeats
X=aDF8.loc[:,RFfeatures].to_numpy(copy=True)    # saving the names
X.shape
y.shape

(2923,)

In [22]:
len(RFfeatures)
X.shape

(2923, 22)

In [23]:
# RFfeatures   # list to scrutinize

In [24]:
XTrain, XTest, yTrain, yTest = train_test_split(X,y, random_state=11)
trainTestData=[XTrain,XTest, yTrain, yTest]
XTrain.shape, XTest.shape

((2192, 22), (731, 22))

In [26]:
yTrain

array([ 68400., 212000., 229000., ..., 122250., 350000., 148000.])

## Do Your Features Vary?

A feature that does not vary, i.e., that has a constant value across all data observations, can't predict anything.  This would be a good place to check all selected features and the target for insufficient variation. Training and test data should be examined.

#### Save Da Data

In [ ]:
RFdb=PickleShareDB('RF.pickleDB')
RFdb.clear()

In [ ]:
RFdb['trainTestData']=trainTestData

## RF Training

Let's train an RF ensemble model using the hyperparameter settings suggested in the Assignment Guide:

* Number of trees = 100
* Maximum number of features = none
* Out of bag (OOB) observations used for validation = True

In [ ]:
RFregr=RandomForestRegressor(max_features=None,oob_score=True,n_jobs=-1,
                            random_state=11)
RFregr.fit(XTrain,yTrain)

In [ ]:
print(f'RF R Squared, Training: {RFregr.score(XTrain,yTrain):5.3f}')
print(f'RF R Squared, OOB: {RFregr.oob_score_:5.3f}')

Let's take a look the $R^2$ based on the test data:

In [ ]:
predTesty=RFregr.predict(XTest)
print(f'Test Data R Squared: {r2_score(yTest,predTesty):4.3f}')

This model definitely needs an intervention.  But just for the sake of demo'ing, let's look at the _importance measures_ for the features.

In [ ]:
RFregr.feature_importances_

We can stick them in a DataFrame, plot them, whatever.

In [ ]:
RFFeatImpDF=pd.DataFrame({'feature':RFfeatures,
                          'importance':RFregr.feature_importances_})

In [ ]:
print('Feature importances')
RFFeatImpDF.sort_values('importance',ascending=False)

See also [permutation feature importance on scikit-learn.org](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance).

## Overfit?

It appears that this model is overfitting the training data.  You could do a grid search to tune various hyperparameters.  Not required for this assignment, but you can do, if you're so inclined.

Here, let's try changing just the maximum number of features used to _log2_, which is a common recommendation when training a RF regressor.

In [ ]:
RFregr2=RandomForestRegressor(max_features='log2',oob_score=True,n_jobs=-1,
                            random_state=11)
RFregr2.fit(XTrain,yTrain)

In [ ]:
print(f'RF R Squared, Training: {RFregr2.score(XTrain,yTrain):4.3f}')
print(f'RF R Squared, OOB: {RFregr2.oob_score_:4.3f}')

In [ ]:
predTesty=RFregr2.predict(XTest)
print(f'Test Data R Squared: {r2_score(yTest,predTesty):4.3f}')

Next, you could try reducing the tree depth, or reducing max_features.  You might also look into increasing the number of trees. What effects do you think each of these changes would have on model fit and generalization?